# Bokeh example
## Setup data

In [1]:
import pandas as pd

In [2]:
votes = pd.read_csv("https://raw.githubusercontent.com/gautsi/gautsi.github.io/master/sketches/carto/nys_ag_dem_pri_votes_032619.csv")

In [3]:
votes.head()

,County,Sean Patrick Maloney (DEM),Letitia A. James (DEM),Leecia R. Eve (DEM),Zephyr Teachout (DEM)
0,Albany County Vote Results,"9,111","4,257","1,716","11,475"
1,Allegany County Vote Results,625,200,66,382
2,Broome County Vote Results,"4,504","1,702",415,"3,710"
3,Cattaraugus County Vote Results,"1,200",318,201,581
4,Cayuga County Vote Results,"1,941",538,103,840


In [4]:
votes_melt = pd.melt(
    votes,
    id_vars=["County"],
    value_vars=[c for c in votes.columns if c != "County"],
    var_name="candidate",
    value_name="num_votes").rename(columns={"County": "county"})

In [5]:
votes_melt["county"] = votes_melt.county.map(lambda x: x.split(" County Vote Results")[0])

In [6]:
votes_melt.head()

,county,candidate,num_votes
0,Albany,Sean Patrick Maloney (DEM),"9,111"
1,Allegany,Sean Patrick Maloney (DEM),625
2,Broome,Sean Patrick Maloney (DEM),"4,504"
3,Cattaraugus,Sean Patrick Maloney (DEM),"1,200"
4,Cayuga,Sean Patrick Maloney (DEM),"1,941"


In [7]:
votes_melt["candidate"] = votes_melt.candidate.map(lambda x: x.split(" (DEM)")[0])

In [8]:
votes_melt["num_votes"] = votes_melt.num_votes.str.replace(",", "").astype("int")

In [9]:
votes_melt.head()

,county,candidate,num_votes
0,Albany,Sean Patrick Maloney,9111
1,Allegany,Sean Patrick Maloney,625
2,Broome,Sean Patrick Maloney,4504
3,Cattaraugus,Sean Patrick Maloney,1200
4,Cayuga,Sean Patrick Maloney,1941


In [10]:
votes_melt_w_win = votes_melt.merge(right=votes_melt.groupby(["county"], as_index=False).num_votes.max().assign(win_cand="Yes"), on = ["county", "num_votes"], how="left").fillna("No")

In [11]:
votes_melt_w_win.head()

,county,candidate,num_votes,win_cand
0,Albany,Sean Patrick Maloney,9111,No
1,Allegany,Sean Patrick Maloney,625,Yes
2,Broome,Sean Patrick Maloney,4504,Yes
3,Cattaraugus,Sean Patrick Maloney,1200,Yes
4,Cayuga,Sean Patrick Maloney,1941,Yes


In [12]:
votes_melt_w_win[votes_melt_w_win.county=="Allegany"]

,county,candidate,num_votes,win_cand
1,Allegany,Sean Patrick Maloney,625,Yes
63,Allegany,Letitia A. James,200,No
125,Allegany,Leecia R. Eve,66,No
187,Allegany,Zephyr Teachout,382,No


In [13]:
counties = votes_melt_w_win.groupby(["county"], as_index=False).num_votes.sum().rename(columns={"num_votes": "ttl_votes"}).merge(
    right=votes_melt_w_win[votes_melt_w_win.win_cand=="Yes"][["county", "candidate"]],
    on=["county"],
    how="left")

In [14]:
counties

,county,ttl_votes,candidate
0,Albany,26559,Zephyr Teachout
1,Allegany,1273,Sean Patrick Maloney
2,Bronx,127398,Letitia A. James
3,Broome,10331,Sean Patrick Maloney
4,Cattaraugus,2300,Sean Patrick Maloney
...,...,...,...
57,Washington,2118,Zephyr Teachout
58,Wayne,2326,Sean Patrick Maloney
59,Westchester,87498,Letitia A. James
60,Wyoming,865,Sean Patrick Maloney


In [16]:
cand = votes_melt_w_win.groupby(["candidate"], as_index=False).num_votes.sum()

In [17]:
cand

,candidate,num_votes
0,Leecia R. Eve,52367
1,Letitia A. James,608308
2,Sean Patrick Maloney,379099
3,Zephyr Teachout,468083


In [18]:
votes_melt_w_win[["county", "candidate", "num_votes"]].to_csv("county_candidates.csv", index=False)

In [19]:
counties.to_csv("counties.csv", index=False)

In [20]:
cand.to_csv("candidates.csv", index=False)